# Ngatamariki Microseismicity Annual Report Data Plots, for Mercury Energy

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

**Import modules, etc**

In [ ]:
#import cartopy
import cartopy.crs as ccrs
#import cartopy.io.img_tiles as cimgt
from cartopy.io.img_tiles import OSM
#from cartopy.io.img_tiles import StamenTerrain
from cartopy.io.img_tiles import QuadtreeTiles

import pandas as pd
import numpy as np
import pyproj
from math import floor

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.transforms import offset_copy
from matplotlib import patheffects
import matplotlib.gridspec as gridspec
import warnings; warnings.simplefilter('ignore')

%matplotlib inline

**Full width notebook display**

In [ ]:
#full width notebook display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Functions

**Array version of Haversine formula to calculate distance between two points**

In [ ]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

**Plot distance scale (and north arrow) on plot. Reference in scale_bar function documentation**

In [ ]:
def utm_from_lon(lon):
    """
    utm_from_lon - UTM zone for a longitude

    Not right for some polar regions (Norway, Svalbard, Antartica)

    :param float lon: longitude
    :return: UTM zone number
    :rtype: int
    """
    return floor( ( lon + 180 ) / 6) + 1

def scale_bar(ax, proj, length, location=(0.5, 0.05), linewidth=1.5,
              units='km', m_per_unit=1000):
    """

    http://stackoverflow.com/a/35705477/1072212
    ax is the axes to draw the scalebar on.
    proj is the projection the axes are in
    location is center of the scalebar in axis coordinates ie. 0.5 is the middle of the plot
    length is the length of the scalebar in km.
    linewidth is the thickness of the scalebar.
    units is the name of the unit
    m_per_unit is the number of meters in a unit
    """
    # find lat/lon center to find best UTM zone
    x0, x1, y0, y1 = ax.get_extent(proj.as_geodetic())
    # Projection in metres
    utm = ccrs.UTM(utm_from_lon((x0+x1)/2))
    # Get the extent of the plotted area in coordinates in metres
    x0, x1, y0, y1 = ax.get_extent(utm)
    # Turn the specified scalebar location into coordinates in metres
    sbcx, sbcy = x0 + (x1 - x0) * location[0], y0 + (y1 - y0) * location[1]
    # Generate the x coordinate for the ends of the scalebar
    bar_xs = [sbcx - length * m_per_unit/2, sbcx + length * m_per_unit/2]
    # buffer for scalebar
    buffer = [patheffects.withStroke(linewidth=3, foreground="w")]
    # Plot the scalebar with buffer
    ax.plot(bar_xs, [sbcy, sbcy], transform=utm, color='k',
        linewidth=linewidth, path_effects=buffer)
    # buffer for text
    buffer = [patheffects.withStroke(linewidth=3, foreground="w")]
    # Plot the scalebar label
    t0 = ax.text(sbcx, sbcy, str(length) + ' ' + units, transform=utm,
        horizontalalignment='center', verticalalignment='bottom',
        path_effects=buffer, zorder=2)
    left = x0+(x1-x0)*0.05
    # Plot the N arrow
    t1 = ax.text(left, sbcy, u'\u25B2\nN', transform=utm,
        horizontalalignment='center', verticalalignment='bottom',
        path_effects=buffer, zorder=2)
    # Plot the scalebar without buffer, in case covered by text buffer
    ax.plot(bar_xs, [sbcy, sbcy], transform=utm, color='k',
        linewidth=linewidth, zorder=3)
    
# or to use m instead of km
# scale_bar(ax, ccrs.Mercator(), 100000, m_per_unit=1, units='m')
# or to use miles instead of km
# scale_bar(ax, ccrs.Mercator(), 60, m_per_unit=1609.34, units='miles')

#need specific font to show triangle for north arrow
mpl.rc('font', family='DejaVu Sans')

## Set Up Parameters

**Plot area**

In [ ]:
lonmin = 176.147
lonmax = 176.243
latmin = -38.586
latmax = -38.510

#northern map extents
northlonmin = 176.164
northlonmax = 176.194
northlatmin = -38.542
northlatmax = -38.525

#southern map extents
southlonmin = 176.181
southlonmax = 176.204
southlatmin = -38.573
southlatmax = -38.552

**Times**

In [ ]:
start = '2012-05-15T00:00:00'
thisrpt = '2017-01-01T00:00:00'
end = '2018-01-01T00:00:00'

#previous report period
lastrpt = '2016-01-01T00:00:00'
lastend = '2017-01-01T00:00:00'

#report period before that
last2rpt = '2015-01-01T00:00:00'
last2end = '2016-01-01T00:00:00'

## Seismographs

In [ ]:
#rotokawa
names = ['lon', 'lat', 'name', 'height']
seisrt = pd.read_csv('seismograph_sites_rotokawa.dat', sep='\s+', names=names)

#ngatamariki
names = ['name', 'lat', 'lon']
seisng = pd.read_csv('seismograph_sites_ngatamariki.csv', names=names)

In [ ]:
seisng.head()

In [ ]:
seisrt.head()

**GeoNet Seismographs**

In [ ]:
names = ['lon', 'lat', 'name']
seisgn = pd.read_csv('geonet_sites.dat', sep='\s+', names=names)

In [ ]:
seisgn

## Seismicity from Local Network

**HypoDD hypocentres for rotnga, first event 2015-05-15**

In [ ]:
#this file needs to be kept up to date as new data are processed
hypdd = pd.read_csv('rotnga_hypodd_all.csv', usecols=[3,4,5,6,7,8], parse_dates={"ot" : ['YYYY-MM-DD','hh:mm:sec']}, index_col='ot')
hypdd.sort_index(inplace=True)

In [ ]:
print (hypdd.head())
print (hypdd.tail())

In [ ]:
len(hypdd)

**Filter for plotting area**

In [ ]:
#filter hypocentres to plot area, to just get ngatamariki
hypdd = hypdd[(hypdd['longitude']>lonmin)&(hypdd['longitude']<lonmax)&(hypdd['latitude']>latmin)&(hypdd['latitude']<latmax)]

In [ ]:
len(hypdd)

In [ ]:
#filter hypocentres for reporting period, to get ngatamariki for just reporting period
hyprpt = hypdd[(hypdd.index>thisrpt)&(hypdd.index<end)]

In [ ]:
len(hyprpt)

In [ ]:
#unset ml shows as -9, which plots as M9 symbol, remove
hyprpt = hyprpt[hyprpt['ml']>0]

In [ ]:
hyprpt['ml'].describe()

In [ ]:
hyprpt[hyprpt['ml']>=3]

In [ ]:
hyprpt[hyprpt['ml']>=2]

**Hypocentre dataframes by magnitude**

In [ ]:
#m1
hyprptm0 = hyprpt[hyprpt['ml']<1]
#m1
hyprptm1 = hyprpt[(hyprpt['ml']>=1)&(hyprpt['ml']<2)]
#m2
hyprptm2 = hyprpt[(hyprpt['ml']>=2)&(hyprpt['ml']<3)]
#m3
hyprptm3 = hyprpt[hyprpt['ml']>=3]

**Plot hypocentres on Quadtree basemap, this is like GoogleEarth**

In [ ]:
#imagery
imagery = QuadtreeTiles()
ax = plt.axes(projection=imagery.crs)
ax.set_extent([lonmin, lonmax, latmin, latmax])
ax.add_image(imagery, 14)
#ax.gridlines()

#hypocentres, symbol coloured is determined by magnitude, with a constant symbol size
#plot in order of increasing magnitude so larger events plot on top
plt.scatter(hyprptm0['longitude'], hyprptm0['latitude'], color='white', marker='o', s=2, transform=ccrs.Geodetic())
plt.scatter(hyprptm1['longitude'], hyprptm1['latitude'], color='yellow', marker='o', s=2, transform=ccrs.Geodetic())
#m2 and m3 may not exist
if len(hyprptm2) != 0:
    plt.scatter(hyprptm2['longitude'], hyprptm2['latitude'], color='darkorange', marker='o', s=2, transform=ccrs.Geodetic())
if len(hyprptm3) != 0:
    plt.scatter(hyprptm3['longitude'], hyprptm3['latitude'], color='red', marker='o', s=2, transform=ccrs.Geodetic())

#plot wells
names = ['lon', 'lat', 'name']
wells = pd.read_csv('wells.dat', sep='\s+', names=names)
plt.scatter(wells['lon'], wells['lat'], color='black', marker='s', s=5, transform=ccrs.Geodetic())
#transform coordinate system so can plot text below symbol
geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
text_transform = offset_copy(geodetic_transform, units='dots', y=-15)
#plot text labels
for index, row in wells.iterrows():
    plt.text(row['lon'], row['lat'], row['name'], verticalalignment='top', horizontalalignment='center', transform=text_transform, fontsize=4)

#seismographs
#ngatamariki
plt.plot(seisng['lon'], seisng['lat'], color='gray', markeredgecolor='black', markeredgewidth=0.2, linestyle='None', marker='^', markersize=4, transform=ccrs.Geodetic())
#rotokawa
plt.plot(seisrt['lon'], seisrt['lat'], color='gray', markeredgecolor='black', markeredgewidth=0.2, linestyle='None', marker='^', markersize=4, transform=ccrs.Geodetic())
#geonet
plt.plot(seisgn['lon'], seisgn['lat'], color='black', markeredgewidth=0, linestyle='None', marker='^', markersize=4, transform=ccrs.Geodetic())
#plot text labels
for index, row in seisng.iterrows():
    plt.text(row['lon'], row['lat'], row['name'], verticalalignment='top', color='white', horizontalalignment='center', transform=text_transform, fontsize=4)
for index, row in seisrt.iterrows():
    plt.text(row['lon'], row['lat'], row['name'], verticalalignment='top', color='white', horizontalalignment='center', transform=text_transform, fontsize=4)
for index, row in seisgn.iterrows():
    plt.text(row['lon'], row['lat'], row['name'], verticalalignment='top', color='white', horizontalalignment='center', transform=text_transform, fontsize=4)
#section line
plt.plot([176.1742,176.2024], [-38.5260,-38.5730], color='black', linestyle='--', marker='None', transform=ccrs.Geodetic())

scale_bar(ax, ccrs.Mercator(), 1)

#legend
#magnitude
m0 = mlines.Line2D([], [], color='white', marker='o', linestyle='None', markersize=5, label=r'$M<1$')
m1 = mlines.Line2D([], [], color='yellow', marker='o', linestyle='None', markersize=5, label=r'$M=1-2$')
m2 = mlines.Line2D([], [], color='darkorange', marker='o', linestyle='None', markersize=5, label=r'$M=2-3$')
m3 = mlines.Line2D([], [], color='red', marker='o', linestyle='None', markersize=5, label=r'$M\geq3$')
#seismographs
ng = mlines.Line2D([], [], color='gray', markeredgecolor='black', markeredgewidth=0.2, linestyle='None', marker='^', markersize=4, label=r'$Mercury seis$')
gn = mlines.Line2D([], [], color='gray', markeredgewidth=0, linestyle='None', marker='^', markersize=4, label=r'$GeoNet seis$')
#wells
wl = mlines.Line2D([], [], color='black', markeredgewidth=0, linestyle='None', marker='s', markersize=4, label=r'$Wells$')
plt.legend(handles=[m0, m1, m2, m3, ng, gn, wl], fontsize='xx-small', loc='upper right', numpoints=1)

plt.savefig('mercury_thisreport_map.png', dpi=400)

**Trim map in bash as matplotlib function does not work**

In [ ]:
%%bash
convert mercury_thisreport_map.png -trim +repage mercury_thisreport_map.png

**Plot northern area**

In [ ]:
#imagery
imagery = QuadtreeTiles()
ax = plt.axes(projection=imagery.crs)
ax.set_extent([northlonmin, northlonmax, northlatmin, northlatmax])
ax.add_image(imagery, 16)

#hypocentres, symbol coloured is determined by magnitude, with a constant symbol size
#plot in order of increasing magnitude so larger events plot on top
plt.scatter(hyprptm0['longitude'], hyprptm0['latitude'], color='white', marker='o', s=2, transform=ccrs.Geodetic())
plt.scatter(hyprptm1['longitude'], hyprptm1['latitude'], color='yellow', marker='o', s=2, transform=ccrs.Geodetic())
#m2 and m3 may not exist
if len(hyprptm2) != 0:
    plt.scatter(hyprptm2['longitude'], hyprptm2['latitude'], color='darkorange', marker='o', s=2, transform=ccrs.Geodetic())
if len(hyprptm3) != 0:
    plt.scatter(hyprptm3['longitude'], hyprptm3['latitude'], color='red', marker='o', s=2, transform=ccrs.Geodetic())

#plot wells
names = ['lon', 'lat', 'name']
wells = pd.read_csv('wellsnorth.dat', sep='\s+', names=names)
plt.scatter(wells['lon'], wells['lat'], color='black', marker='s', s=5, transform=ccrs.Geodetic())
#transform coordinate system so can plot text below symbol
geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
text_transform = offset_copy(geodetic_transform, units='dots', y=-15)
#plot text labels
for index, row in wells.iterrows():
    plt.text(row['lon'], row['lat'], row['name'], verticalalignment='top', horizontalalignment='center', transform=text_transform, fontsize=5)

#seismographs
#ngatamariki
plt.plot(seisng['lon'], seisng['lat'], color='gray', markeredgecolor='black', markeredgewidth=0.2, linestyle='None', marker='^', markersize=4, transform=ccrs.Geodetic())
#geonet
plt.plot(seisgn['lon'], seisgn['lat'], color='black', markeredgewidth=0, linestyle='None', marker='^', markersize=4, transform=ccrs.Geodetic())
#plot text labels
for index, row in seisng.iterrows():
    plt.text(row['lon'], row['lat'], row['name'], verticalalignment='top', color='white', horizontalalignment='center', transform=text_transform, fontsize=4)
for index, row in seisgn.iterrows():
    plt.text(row['lon'], row['lat'], row['name'], verticalalignment='top', color='white', horizontalalignment='center', transform=text_transform, fontsize=4)

#legend
#magnitude
m0 = mlines.Line2D([], [], color='white', marker='o', linestyle='None', markersize=5, label=r'$M<1$')
m1 = mlines.Line2D([], [], color='yellow', marker='o', linestyle='None', markersize=5, label=r'$M=1-2$')
m2 = mlines.Line2D([], [], color='darkorange', marker='o', linestyle='None', markersize=5, label=r'$M=2-3$')
m3 = mlines.Line2D([], [], color='red', marker='o', linestyle='None', markersize=5, label=r'$M\geq3$')
#seismographs
ng = mlines.Line2D([], [], color='gray', markeredgecolor='black', markeredgewidth=0.2, linestyle='None', marker='^', markersize=4, label=r'$Mercury seis$')
gn = mlines.Line2D([], [], color='gray', markeredgewidth=0, linestyle='None', marker='^', markersize=4, label=r'$GeoNet seis$')
#wells
wl = mlines.Line2D([], [], color='black', markeredgewidth=0, linestyle='None', marker='s', markersize=4, label=r'$Wells$')
plt.legend(handles=[m0, m1, m2, m3, ng, gn, wl], fontsize='xx-small', loc='upper right', numpoints=1)

scale_bar(ax, ccrs.Mercator(), 1)

plt.savefig('mercury_thisreport_map_north.png', dpi=400)

**Trim map in bash as matplotlib function does not work**

In [ ]:
%%bash
convert mercury_thisreport_map_north.png -trim +repage mercury_thisreport_map_north.png

**Plot southern area**

In [ ]:
#imagery
imagery = QuadtreeTiles()
ax = plt.axes(projection=imagery.crs)
ax.set_extent([southlonmin, southlonmax, southlatmin, southlatmax])
ax.add_image(imagery, 16)

#hypocentres, symbol coloured is determined by magnitude, with a constant symbol size
#plot in order of increasing magnitude so larger events plot on top
plt.scatter(hyprptm0['longitude'], hyprptm0['latitude'], color='white', marker='o', s=2, transform=ccrs.Geodetic())
plt.scatter(hyprptm1['longitude'], hyprptm1['latitude'], color='yellow', marker='o', s=2, transform=ccrs.Geodetic())
#m2 and m3 may not exist
if len(hyprptm2) != 0:
    plt.scatter(hyprptm2['longitude'], hyprptm2['latitude'], color='darkorange', marker='o', s=2, transform=ccrs.Geodetic())
if len(hyprptm3) != 0:
    plt.scatter(hyprptm3['longitude'], hyprptm3['latitude'], color='red', marker='o', s=2, transform=ccrs.Geodetic())

#plot wells
names = ['lon', 'lat', 'name']
wells = pd.read_csv('wellssouth.dat', sep='\s+', names=names)
plt.scatter(wells['lon'], wells['lat'], color='black', marker='s', s=5, transform=ccrs.Geodetic())
#transform coordinate system so can plot text below symbol
geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
text_transform = offset_copy(geodetic_transform, units='dots', y=-15)
#plot text labels
for index, row in wells.iterrows():
    plt.text(row['lon'], row['lat'], row['name'], verticalalignment='top', horizontalalignment='center', transform=text_transform, fontsize=4)
#plot deviated well NM10
names = ['lon', 'lat', 'dep']
nm10 = pd.read_csv('well_nm10.dat', sep='\s+', names=names)
plt.plot(nm10['lon'], nm10['lat'], color='black', linestyle='-', marker='None', transform=ccrs.Geodetic())

#seismographs
#ngatamariki
plt.plot(seisng['lon'], seisng['lat'], color='gray', markeredgecolor='black', markeredgewidth=0.2, linestyle='None', marker='^', markersize=4, transform=ccrs.Geodetic())
#geonet
plt.plot(seisgn['lon'], seisgn['lat'], color='black', markeredgewidth=0, linestyle='None', marker='^', markersize=4, transform=ccrs.Geodetic())
#plot text labels
for index, row in seisng.iterrows():
    plt.text(row['lon'], row['lat'], row['name'], verticalalignment='top', color='white', horizontalalignment='center', transform=text_transform, fontsize=4)
for index, row in seisgn.iterrows():
    plt.text(row['lon'], row['lat'], row['name'], verticalalignment='top', color='white', horizontalalignment='center', transform=text_transform, fontsize=4)

#legend
#magnitude
m0 = mlines.Line2D([], [], color='white', marker='o', linestyle='None', markersize=5, label=r'$M<1$')
m1 = mlines.Line2D([], [], color='yellow', marker='o', linestyle='None', markersize=5, label=r'$M=1-2$')
m2 = mlines.Line2D([], [], color='darkorange', marker='o', linestyle='None', markersize=5, label=r'$M=2-3$')
m3 = mlines.Line2D([], [], color='red', marker='o', linestyle='None', markersize=5, label=r'$M\geq3$')
#seismographs
ng = mlines.Line2D([], [], color='gray', markeredgecolor='black', markeredgewidth=0.2, linestyle='None', marker='^', markersize=4, label=r'$Mercury seis$')
gn = mlines.Line2D([], [], color='gray', markeredgewidth=0, linestyle='None', marker='^', markersize=4, label=r'$GeoNet seis$')
#wells
wl = mlines.Line2D([], [], color='black', markeredgewidth=0, linestyle='None', marker='s', markersize=4, label=r'$Wells$')
plt.legend(handles=[m0, m1, m2, m3, ng, gn, wl], fontsize='xx-small', loc='upper right', numpoints=1)
    
scale_bar(ax, ccrs.Mercator(), 1)

plt.savefig('mercury_thisreport_map_south.png', dpi=400)

**Trim map in bash as matplotlib function does not work**

In [ ]:
%%bash
convert mercury_thisreport_map_south.png -trim +repage mercury_thisreport_map_south.png

**Cross-section, within 1.5 km of the line**

In [ ]:
#export data
hyprpt.to_csv('section.csv', header=False)

#run external program, temporary solution
!section.csh

In [ ]:
#read output of external program
names=['p', 'q', 'z', 'ml']
seceqs = pd.read_csv('section.dat', sep='\s+', names=names)
#pq nomerclature is that used by GMT project command http://gmt.soest.hawaii.edu/doc/5.4.2/project.html

In [ ]:
seceqs.head()

**By magnitude**

In [ ]:
#m1
secm0 = seceqs[seceqs['ml']<1]
#m1
secm1 = seceqs[(seceqs['ml']>=1)&(seceqs['ml']<2)]
#m2
secm2 = seceqs[(seceqs['ml']>=2)&(seceqs['ml']<3)]
#m3
secm3 = seceqs[seceqs['ml']>=3]

In [ ]:
#read well data (pre-projected using well_section.csh)
names=['p', 'q', 'z']
well01 = pd.read_csv('well_section_01.dat', sep='\s+', names=names)
well02 = pd.read_csv('well_section_02.dat', sep='\s+', names=names)
well03 = pd.read_csv('well_section_03.dat', sep='\s+', names=names)
well04 = pd.read_csv('well_section_04.dat', sep='\s+', names=names)
well05 = pd.read_csv('well_section_05.dat', sep='\s+', names=names)
well06 = pd.read_csv('well_section_06.dat', sep='\s+', names=names)
well07 = pd.read_csv('well_section_07.dat', sep='\s+', names=names)
well08 = pd.read_csv('well_section_08.dat', sep='\s+', names=names)
well09 = pd.read_csv('well_section_09.dat', sep='\s+', names=names)
well10 = pd.read_csv('well_section_10.dat', sep='\s+', names=names)
well11 = pd.read_csv('well_section_11.dat', sep='\s+', names=names)
well12 = pd.read_csv('well_section_12.dat', sep='\s+', names=names)

In [ ]:
#size scaled to give 1:1 ratio
fig = plt.figure(figsize=(6,5.3))

#earthquakes
#plt.scatter(x=seceqs['p'], y=seceqs['z'], s=2*seceqs['ml']**2, color='red', marker='o', linewidth=0.3)
#hypocentres, symbol coloured is determined by magnitude, with a constant symbol size
#plot in order of increasing magnitude so larger events plot on top
plt.scatter(secm0['p'], secm0['z'], color='white', edgecolors='black', marker='o', s=4, linewidth=0.3)
plt.scatter(secm1['p'], secm1['z'], color='yellow', marker='o', s=4)
#m2 and m3 may not exist
if len(secm2) != 0:
    plt.scatter(secm2['p'], secm2['z'], color='darkorange', marker='o', s=4)
if len(secm3) != 0:
    plt.scatter(secm3['p'], secm3['z'], color='red', marker='o', s=4)
# wells
plt.plot(well01['p'], well01['z'], color='black')
plt.plot(well02['p'], well02['z'], color='black')
plt.plot(well03['p'], well03['z'], color='black')
plt.plot(well04['p'], well04['z'], color='black')
plt.plot(well05['p'], well05['z'], color='black')
plt.plot(well06['p'], well06['z'], color='black')
plt.plot(well07['p'], well07['z'], color='black')
plt.plot(well08['p'], well08['z'], color='black')
plt.plot(well09['p'], well09['z'], color='black')
plt.plot(well10['p'], well10['z'], color='black')
plt.plot(well11['p'], well11['z'], color='black')
plt.plot(well12['p'], well12['z'], color='black')
#well labels
plt.text(-2.1, 0.1, 'NM01', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=8)
plt.text(-1.45, 0.1, 'NM02', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=8)
plt.text(-0.85, 0.1, 'NM03', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=8)
plt.text(-2.7, 0.1, 'NM04', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=8)
plt.text(0.9, 0.1, 'NM05', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=8)
plt.text(2.05, 0.1, 'NM06', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=8)
plt.text(-0.4, 0.1, 'NM07', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=8)
plt.text(-2.1, 0.5, 'NM08', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=8)
plt.text(-1.95, 0.1, 'NM09', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=8)
plt.text(1.9, 0.1, 'NM10', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=8)
plt.text(0.75, 0.1, 'NM11', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=8)
plt.text(0.2, 0.1, 'NM12', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=8)
#section orientations
plt.text(-2.9, -0, 'N', verticalalignment='bottom', horizontalalignment='left', fontsize=10)
plt.text(2.9, -0, 'S', verticalalignment='bottom', horizontalalignment='right', fontsize=10)

#legend
#magnitude
m0 = mlines.Line2D([], [], color='white', marker='o', markeredgecolor='black', linestyle='None', markersize=3, label=r'$M<1$')
m1 = mlines.Line2D([], [], color='yellow', marker='o', linestyle='None', markersize=3, label=r'$M=1-2$')
m2 = mlines.Line2D([], [], color='darkorange', marker='o', linestyle='None', markersize=3, label=r'$M=2-3$')
m3 = mlines.Line2D([], [], color='red', marker='o', linestyle='None', markersize=3, label=r'$M\geq3$')
plt.legend(handles=[m0, m1, m2, m3], fontsize='xx-small', loc='lower right', numpoints=1)

plt.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
plt.xlim([-3,3])
plt.ylim([-0.3,5])
plt.ylabel('Depth (km)')
plt.xlabel('Distance from field "centre" (km)')
plt.gca().invert_yaxis()
plt.tight_layout()

plt.savefig('mercury_thisreport_section.png', dpi=200)

**Magnitude-time - Reporting period only**

In [ ]:
from datetime import datetime

#divide into north and south clusters, latitude is that on power station
hyprptnorth = hyprpt[hyprpt['latitude']>-38.546]
hyprptsouth = hyprpt[hyprpt['latitude']<-38.546]

#magnitude vs time
ax = hyprptnorth.plot(y='ml', color='red', marker='o', linewidth=0.3, linestyle='None', label='north', figsize=(10,5))
hyprptsouth.plot(ax=ax, y='ml', color='black', marker='o', linewidth=0.3, linestyle='None', label='south')
ax.xaxis.grid()
ax.yaxis.grid()

ax.set_ylim([0,3.0])
ax.set_xlim([thisrpt,end])
ax.set_ylabel('Magnitude')
ax.tick_params(labelright = True)

plt.xlabel("") #get rid of useless 'date' label
plt.legend(loc='best')
plt.tight_layout()
plt.savefig('mercury_thisreport_time-series.png', dpi=200)

In [ ]:
print (len(hyprptnorth))
print (len(hyprptsouth))

**Total events per year, divided into north and south**

In [ ]:
#divide into north and south clusters, latitude is that of power station
#hypdd is events within Ngatamariki area

hypdd = hypdd[hypdd.index < end]

north = hypdd[hypdd['latitude']>-38.546]
south = hypdd[hypdd['latitude']<-38.546]

annualnorth = north['latitude'].groupby([north.index.year]).agg('count')
annualsouth = south['latitude'].groupby([south.index.year]).agg('count')

annual = pd.concat([annualnorth, annualsouth], axis=1)
annual.columns=['north', 'south']

In [ ]:
an = annual.plot.bar(color=['red', 'black'])
an.set_ylabel('Number per year')
an.set_xlabel('Year')

fig = an.get_figure()
fig.savefig('annual_meqs.png', dpi=200)

**Plot wide area station map, showing Rotokawa and GeoNet**

**Station map plot area**

In [ ]:
wlonmin = 176.116
wlonmax = 176.419
wlatmin = -38.678
wlatmax = -38.370

In [ ]:
#imagery
imagery = OSM()
ax = plt.axes(projection=imagery.crs)
ax.set_extent([wlonmin, wlonmax, wlatmin, wlatmax])
ax.add_image(imagery, 10)
#ax.gridlines()

#extents of detailed plot area
boxlon = [lonmin, lonmin, lonmax, lonmax, lonmin]
boxlat = [latmin, latmax, latmax, latmin, latmin]
plt.plot(boxlon, boxlat, color='black', linestyle='-', marker='None', transform=ccrs.Geodetic())

#seismographs ngatamariki
plt.plot(seisng['lon'], seisng['lat'], color='gray', markeredgewidth=0.2, linestyle='None', marker='^', markersize=3, transform=ccrs.Geodetic())
geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
text_transform = offset_copy(geodetic_transform, units='dots', y=-10)
#plot text labels
for index, row in seisng.iterrows():
    plt.text(row['lon'], row['lat'], row['name'], verticalalignment='top', horizontalalignment='center', transform=text_transform, fontsize=3)
#seismographs rotokawa
plt.plot(seisrt['lon'], seisrt['lat'], color='gray', markeredgewidth=0.0, linestyle='None', marker='o', markersize=3, transform=ccrs.Geodetic())
#seismographs geonet
plt.plot(seisgn['lon'], seisgn['lat'], color='black', markeredgewidth=0.0, linestyle='None', marker='o', markersize=4, transform=ccrs.Geodetic())

#transform coordinate system so can plot text below symbol
geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
text_transform = offset_copy(geodetic_transform, units='dots', y=0, x=0)
plt.text(176.200, -38.546, 'N', verticalalignment='center', horizontalalignment='center', transform=text_transform, fontsize=6)
plt.text(176.200, -38.625, 'R', verticalalignment='center', horizontalalignment='center', transform=text_transform, fontsize=6)

#legend
ng = mlines.Line2D([], [], color='gray', markeredgecolor='black', markeredgewidth=0.2, linestyle='None', marker='^', markersize=3, label='Ngatamariki')
rt = mlines.Line2D([], [], color='gray', markeredgewidth=0, linestyle='None', marker='o', markersize=3, label='Rotokawa')
gn = mlines.Line2D([], [], color='black', markeredgewidth=0, linestyle='None', marker='o', markersize=3, label='GeoNet')

plt.legend(handles=[ng, rt, gn], fontsize='xx-small', loc='upper left', numpoints=1)

scale_bar(ax, ccrs.Mercator(), 5)
 
plt.savefig('mercury_wide-station_map.png', dpi=400)

**Trim map in bash as matplotlib function does not work**

In [ ]:
%%bash
convert mercury_wide-station_map.png -trim +repage mercury_wide-station_map.png

## GeoNet seismicity

In [ ]:
#box for search area, this is plenty big enough to les get events within 10 km of reference point
bbox = '176.0,-38.8,176.5,-38.3'
maxdep = '20'

url = 'https://quakesearch.geonet.org.nz/csv?bbox='+bbox+'&maxdepth='+maxdep+'&startdate='+start+'&enddate='+end

In [ ]:
eqs = pd.read_csv(url, parse_dates=['origintime'], index_col='origintime', usecols=[2,4,5,6,7])

In [ ]:
eqs.head()

In [ ]:
#reference point (Ngatamariki power station)
latref = -38.546143
lonref = 176.195504

In [ ]:
eqs['dist'] = haversine_np(eqs['longitude'],eqs[' latitude'],lonref,latref)

In [ ]:
eqs10 = eqs[eqs['dist']<10]

In [ ]:
eqs10['dist'].describe()

In [ ]:
eqs10.head()

In [ ]:
eqs10[eqs10[' magnitude']>=3]

**Number of GeoNet earthquakes within 5 & 10 km during the report period**

In [ ]:
eqs[' magnitude'][(eqs['dist']<=10)&(eqs.index>=thisrpt)&(eqs.index<=end)].describe()

In [ ]:
eqs[' magnitude'][(eqs['dist']<=5)&(eqs.index>=thisrpt)&(eqs.index<=end)].describe()

**And for the previous report period**

In [ ]:
eqs[' magnitude'][(eqs['dist']<=10)&(eqs.index>=lastrpt)&(eqs.index<=lastend)].describe()

In [ ]:
eqs[' magnitude'][(eqs['dist']<=5)&(eqs.index>=lastrpt)&(eqs.index<=lastend)].describe()

**And the one before that**

In [ ]:
eqs[' magnitude'][(eqs['dist']<=10)&(eqs.index>=last2rpt)&(eqs.index<=last2end)].describe()

In [ ]:
eqs[' magnitude'][(eqs['dist']<=5)&(eqs.index>=last2rpt)&(eqs.index<=last2end)].describe()

**Biggest events**

In [ ]:
eqs[eqs[' magnitude']>3.5]

**Plots vs Time**

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2,ncols=1, sharex=True, figsize=(22,10))

#distance vs time
eqs10.plot(y='dist', marker='o', markersize=5, linestyle='None', color='black', legend=False, ax=ax1)

#plot label A
ax1.text('2012-06-01', 9, 'A', size=16)

#yearly ticks and grid
majorTick = mpl.dates.YearLocator(1)
majorFormat = mpl.dates.DateFormatter('%Y')
ax1.xaxis.set_major_locator(majorTick)
ax1.xaxis.set_major_formatter(majorFormat)
ax1.xaxis.grid()
ax1.yaxis.grid()

ax1.set_ylabel('Distance (km)')

ax1.tick_params(labelright = True)

#dashed line at dist=5 km
ax1.axhline(y=5, color='blue', linestyle='--')

#magnitude vs time
eqs10.plot(y=' magnitude', marker='o', markersize=5, linestyle='None', color='black', legend=False, ax=ax2)

#plot label 8
ax2.text('2012-06-01', 3.7, 'B', size=16)

#yearly ticks and grid
ax2.xaxis.set_major_locator(majorTick)
ax2.xaxis.set_major_formatter(majorFormat)
ax2.xaxis.grid()
ax2.yaxis.grid()

ax2.set_ylim([0,4.2])
ax2.set_xlim(start, end)
ax2.set_ylabel('Magnitude')
ax2.tick_params(labelright = True)

#dashed line at magnitude=2, about felt threshold
ax2.axhline(y=2, color='blue', linestyle='--')

#indicate this report
ax2.axvline(x=thisrpt, color='blue', linestyle='--')
ax2.text(thisrpt, 0.5, 'this report', color='blue')
ax2.arrow(thisrpt, 0.7, 100, 0, head_width=0.1, head_length=20, fc='blue', ec='blue')

plt.tight_layout()
plt.savefig('geonet_long-term_time-series.png', dpi=200)